# Spark SQL Window functions their syntax and how to use them with aggregate function.

In [1]:


#Importing reuired libraries
import datetime as dt
import json
import pyspark
from pyspark.sql import functions as f

from pyspark.sql.functions import *

from pyspark.sql.types import *
from pyspark.sql import HiveContext,SparkSession

#Creating Spark Session 
spark = SparkSession.builder\
        .master("local")\
        .enableHiveSupport()\
        .appName("spark-sql-window-functions")\
        .config("hive.exec.dynamic.partition", "true")\
        .config("hive.exec.dynamic.partition.mode", "nonstrict")\
        .config("spark.sql.warehouse.dir", "warehehousedir")\
        .getOrCreate()

In [2]:
df = spark.read.option("header",True).option("inferSchema",True).option("delimiter",",")\
   .csv("file://///home/abhijeet/Desktop/employee_data.csv")
df.printSchema()
df.registerTempTable('u')
df.show()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
|          Sam|     Sales|  3000|
|        Peter|     Sales|  4600|
|       Sandra|     Sales|  4100|
|         Alex|   Finance|  3000|
|          Jef|     Sales|  3000|
|          Pat|   Finance|  3300|
|        Marko|   Finance|  3900|
|         John| Marketing|  3000|
|         Joss| Marketing|  2000|
|        Jason|     Sales|  4100|
+-------------+----------+------+



row_number() window function is used to give the sequential row number,
starting from 1 to the result of each window partition.

In [3]:

from pyspark.sql import Window
windowSpec  = Window.orderBy("salary")
df.withColumn("row_number",f.row_number().over(windowSpec)).show()
print(df)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        Kumar| Marketing|  2000|         1|
|         Joss| Marketing|  2000|         2|
|        James|     Sales|  3000|         3|
|        Maria|   Finance|  3000|         4|
|        James|     Sales|  3000|         5|
|         Jeff| Marketing|  3000|         6|
|          Sam|     Sales|  3000|         7|
|         Alex|   Finance|  3000|         8|
|          Jef|     Sales|  3000|         9|
|         John| Marketing|  3000|        10|
|        Scott|   Finance|  3300|        11|
|          Pat|   Finance|  3300|        12|
|          Jen|   Finance|  3900|        13|
|        Marko|   Finance|  3900|        14|
|       Robert|     Sales|  4100|        15|
|         Saif|     Sales|  4100|        16|
|       Sandra|     Sales|  4100|        17|
|        Jason|     Sales|  4100|        18|
|      Michael|     Sales|  4600|        19|
|        P

how to calculate sum, min, max for each department using Spark SQL Aggregate window functions and WindowSpec. 
When working with Aggregate functions, we don’t need to use order by clause.

In [5]:

windowSpec=Window.orderBy("salary")
df.withColumn("row_number",f.row_number().over(windowSpec)).withColumn("avg", avg(col("salary")).over(windowSpec))\
    .withColumn("sum", sum(col("salary")).over(windowSpec))\
    .withColumn("min", min(col("salary")).over(windowSpec))\
    .withColumn("max", max(col("salary")).over(windowSpec))\
    .where(col("department")=="Sales").select("department","avg","sum","min","max")\
    .show()

+----------+------------------+-----+----+----+
|department|               avg|  sum| min| max|
+----------+------------------+-----+----+----+
|     Sales|            2800.0|28000|2000|3000|
|     Sales|            2800.0|28000|2000|3000|
|     Sales|            2800.0|28000|2000|3000|
|     Sales|            2800.0|28000|2000|3000|
|     Sales|3266.6666666666665|58800|2000|4100|
|     Sales|3266.6666666666665|58800|2000|4100|
|     Sales|3266.6666666666665|58800|2000|4100|
|     Sales|3266.6666666666665|58800|2000|4100|
|     Sales|            3400.0|68000|2000|4600|
|     Sales|            3400.0|68000|2000|4600|
+----------+------------------+-----+----+----+

